In [ ]:
import datetime
from pathlib import Path

import numpy as np
import polars as pl
import matplotlib.pyplot as plt

import stock

In [ ]:
## 過去データで再現性テスト
### いくつかの局面でrelative strengthがうまく機能するかチェックする
"""
- コロナによる急落局面 (2020)
- そこからの回復局面 (2020 ~ 2021)
- 利上げによる下落局面(2022)
- 生成AIブームによる上昇局面 (2023~2024)
"""

In [ ]:
def get_performance(code: pl.DataFrame, start_date: datetime.date, end_date: datetime.date):
    df = stock.kabutan.read_data_csv(
        stock.PROJECT_ROOT / "data" / "daily" / f"{code}.csv", start_date=start_date, end_date=end_date)
    gain_rate = 0.0
    min_val = 1e9
    max_val = 0
    for i in range(len(df)):
        close_val = df["close"][i]
        if close_val < min_val:
            gain_rate = max(max_val / min_val - 1.0, gain_rate)
            min_val = close_val
            max_val = close_val

        if close_val > max_val:
            max_val = close_val
    return gain_rate

# 値上がり銘柄の取得
def get_high_performe_codes(start_date: datetime.date, end_date: datetime.date):
    """
    """
    csv_dir = stock.PROJECT_ROOT / "data/daily"
    performances = {}
    for csv_path in sorted(csv_dir.glob("*.csv")):
        code = csv_path.stem
        performances[code] = get_performance(code, start_date, end_date)
    return performances

In [ ]:
# 2020/2 ~ 
start_date = datetime.date(year=2020, month=2, day=1)
end_date = datetime.date(year=2021, month=2, day=1)
performances = get_high_performe_codes(start_date=start_date, end_date=end_date)

In [ ]:
{code: val for code, val in performances.items() if val > 0.5}

In [ ]:
code = "4446"
df = stock.kabutan.read_data_csv(csv_path=stock.PROJECT_ROOT / f"data/daily/{code}.csv")
ref_df = stock.kabutan.read_data_csv(csv_path=stock.PROJECT_ROOT / "data/daily/0010.csv")
df = stock.trend_template.calc_rs_v2(df, ref_df)
stock.visualize.plot_with_rs(df=df, start_date=start_date - datetime.timedelta(days=120), end_date=end_date)

In [ ]:
df.filter(pl.col("date").is_between(
    datetime.date(year=2020, month=1, day=1), 
    datetime.date(year=2020, month=2, day=1)
))

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=df['date'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])
fig.update_xaxes(
   rangebreaks=[
     dict(bounds=["sat", "mon"]),
   ]
 )
fig.show()